In [185]:
import pandas as pd
import numpy as np
from PIL import Image
import cv2
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D,GlobalMaxPooling2D
from keras.applications import MobileNet,VGG16
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam,SGD,RMSprop
from sklearn.model_selection import train_test_split

In [186]:
train_data_dir = "./ANIMALS/train/"
test_data_dir = "./ANIMALS/test/"
np_epochs = 5
lr = 1e-3
dc = lr/np_epochs
mom = 0.6
batch = 32

In [187]:
base_model=MobileNet(weights='imagenet',include_top=False)

x=base_model.output
x=GlobalMaxPooling2D()(x)
x=Dense(1024,activation='relu')(x)
x=Dense(512,activation='relu')(x)
preds=Dense(3,activation='softmax')(x)

In [188]:
model=Model(inputs=base_model.input,outputs=preds)

In [189]:
for layer in model.layers[:80]:
    layer.trainable=False
for layer in model.layers[80:]:
    layer.trainable=True

In [190]:
train_datagen = ImageDataGenerator(rotation_range=40,
                                   shear_range = 0.4,
                                   zoom_range = 0.4,
                                   horizontal_flip=True,
                                   preprocessing_function= preprocess_input,
                                   validation_split=0.2)

In [191]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (224,224),
    batch_size = batch,
    class_mode = 'categorical',
    subset='training',
    seed = 48)

Found 801 images belonging to 3 classes.


In [192]:
valid_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (224,224),
    batch_size = batch,
    class_mode = 'categorical',
    subset='training',
    seed = 48)

Found 801 images belonging to 3 classes.


In [193]:
opt = SGD(learning_rate = lr,decay=dc,momentum=mom)
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

In [194]:
model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch,
    validation_data = valid_generator, 
    validation_steps = valid_generator.samples // batch,
    epochs = np_epochs)

Epoch 1/5
25/25 [==============================] - 53s 2s/step - loss: 1.7965 - accuracy: 0.5129 - val_loss: 0.2000 - val_accuracy: 0.9275
Epoch 2/5
25/25 [==============================] - 49s 2s/step - loss: 1.4422 - accuracy: 0.7720 - val_loss: 0.1962 - val_accuracy: 0.9312
Epoch 3/5
25/25 [==============================] - 48s 2s/step - loss: 0.2640 - accuracy: 0.9264 - val_loss: 0.1567 - val_accuracy: 0.9475
Epoch 4/5
25/25 [==============================] - 56s 2s/step - loss: 0.6575 - accuracy: 0.8169 - val_loss: 0.1480 - val_accuracy: 0.9425
Epoch 5/5
25/25 [==============================] - 45s 2s/step - loss: 0.1832 - accuracy: 0.9247 - val_loss: 0.1144 - val_accuracy: 0.9650


In [195]:
model.save("DOGCATPANDAS.h5")

In [196]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = test_datagen.flow_from_directory(test_data_dir,
                                                  target_size =(224,224),seed = 48)

Found 600 images belonging to 3 classes.


In [197]:
scores = model.evaluate_generator(test_generator);

In [198]:
print("Accuracy: " + str(100*scores[1]))

Accuracy: 94.16666626930237
